In [1]:
# 1. 导入依赖 & 读取数据
import pandas as pd

# 载入明细和标签，字段名与 featureprocessing-Copy1.ipynb 保持一致
df = pd.read_csv('model1_data.csv', encoding='gbk', low_memory=False)
dflabel = pd.read_csv('model1_label.csv', encoding='gbk')

# 重命名字段并合并标签
df.rename(columns={
    '卡号': 'card_id',
    '机构名称': 'org_id',
    '结算日期时间': 'settle_time',
    '明细项目交易费用': 'fee',
}, inplace=True)

dflabel.rename(columns={'卡号': 'card_id', '标签': 'label'}, inplace=True)

df = pd.merge(df, dflabel, on='card_id', how='inner')

print(df[['card_id', '明细项目名称']].head())

                                card_id       明细项目名称
0  ff88846b-56ec-4d2f-a2fc-aca3c116c865        尼可地尔片
1  ff88846b-56ec-4d2f-a2fc-aca3c116c865  头孢克洛缓释片(II)
2  ff88846b-56ec-4d2f-a2fc-aca3c116c865      乳果糖口服溶液
3  ff88846b-56ec-4d2f-a2fc-aca3c116c865        艾司唑仑片
4  dccc6fc4-d367-420f-846d-ef5ece5cc1d2    盐酸地尔硫卓缓释片


In [2]:
# 2. 构造账户级“项目文档”并计算 TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# 去除缺失项目名称，只保留 card_id 和 明细项目名称
df_items = df.dropna(subset=['明细项目名称'])[['card_id', '明细项目名称']]

# 按账户聚合为项目列表
card_items = (
    df_items
    .groupby('card_id')['明细项目名称']
    .apply(list)
)

# 构造“文档字符串”：每个账户的项目序列，用空格连接
card_docs = card_items.apply(lambda items: ' '.join(map(str, items)))

print('账户数:', len(card_docs))
print(card_docs.head())

# 以完整项目名称为 token，不拆分中文
vectorizer = TfidfVectorizer(
    analyzer='word',
    token_pattern=r'[^ ]+',  # 按空格分词
    min_df=2,                # 至少在 2 个账户中出现
)

tfidf_matrix = vectorizer.fit_transform(card_docs.values)
print('TF-IDF 形状:', tfidf_matrix.shape)

card_ids = card_docs.index.to_list()

账户数: 8917
card_id
00022092-02fc-45e0-83f2-c51a0d02f2d0    赤小豆 芡实 白茯苓 人参片 薏苡仁 百乐眠胶囊 拉坦前列素滴眼液 马来酸噻吗洛尔滴眼液 芪...
000e9b7e-6a96-4eda-947b-425e964e1212    甲磺酸多沙唑嗪缓释片 宁泌泰胶囊 氯化钠注射液 丹红注射液 氯化钠注射液 盐酸倍他司汀注射液...
000f8286-aa23-42d7-8510-2fab100bcc7b    硝苯地平控释片 胞磷胆碱钠片 盐酸舍曲林片 胞磷胆碱钠片 丁丙诺啡透皮贴剂 金水宝片 银杏叶...
00117f6c-e739-4913-b453-85a118a47123    宣肺止嗽合剂 左氧氟沙星片 复方丹参滴丸 迈之灵片 头孢克洛缓释片 吡格列酮二甲双胍片 消渴...
001c5c03-1db7-4303-934e-21decf219ab1    参松养心胶囊 麝香保心丸 利伐沙班片 维生素B2片 双歧杆菌三联活菌胶囊 胰激肽原酶肠溶片 ...
Name: 明细项目名称, dtype: object
TF-IDF 形状: (8917, 3385)


In [6]:
# 3. 基于 TF-IDF 向量训练 One-Class SVM（OCSVM）
from sklearn.svm import OneClassSVM

# 注意：OCSVM 对大规模高维稀疏矩阵比较敏感，
# 如内存或速度有问题，可以先用 TruncatedSVD 降维后再训练。

ocs = OneClassSVM(
    kernel='rbf',
    gamma='scale',   # 自动按 1 / (n_features * X.var()) 设置
    nu=0.2,         # 预期异常比例（上界），可根据业务调整
)

# OneClassSVM 需要稠密矩阵，转换为 array（如太大可考虑降维）
X = tfidf_matrix.toarray()

ocs.fit(X)

# decision_function：越小越异常
anomaly_scores = ocs.decision_function(X)

# predict：-1 为异常，1 为正常
pred_labels = ocs.predict(X)

import pandas as pd
print('预测标签分布:', pd.Series(pred_labels).value_counts())

预测标签分布:  1    7124
-1    1793
Name: count, dtype: int64


In [7]:
# 4. 结果整理与导出

result_df = pd.DataFrame({
    'card_id': card_ids,
    'ocsvm_label': pred_labels,     # -1 异常，1 正常
    'ocsvm_score': anomaly_scores,  # 越小越异常
})

# 按异常分数从小到大排序（最异常的在最上面）
result_df = result_df.sort_values('ocsvm_score')

# 保存结果
result_df.to_csv('ocsvm_tfidf_results.csv', index=False, encoding='utf-8')

result_df.head(10)

,card_id,ocsvm_label,ocsvm_score
4064,77204f7e-9a0f-4688-9b82-288999f62300,-1,-7.622932
6630,c09fb8bf-450f-4c9a-9c52-d221ea9e9b46,-1,-7.622932
4634,880ac85f-bbc2-4f97-bb80-cf7a511f65e5,-1,-7.572164
5206,9797a83a-14b9-4784-a6ef-44fb56f529f4,-1,-7.519828
5639,a34b0f72-c430-435a-bcde-bbf86ba5b16e,-1,-7.477058
8757,fb3e7f87-d5da-4b9c-8608-779e50cd5e0a,-1,-7.323199
1421,2a2ba3eb-1a49-48e1-82da-db9ac596330d,-1,-7.266801
7586,d9c5e089-26a6-44c2-8c8e-5e701f9ebfce,-1,-7.195720
7342,d2c69ded-fe3c-45df-a581-bcd3a40b1af3,-1,-7.167015
5946,ac2ccc2d-bff9-4e30-bcb5-721e17619c28,-1,-7.087858


In [8]:
# 5. 计算评估指标：AUC、PR-AUC、Precision、Recall、F1

from sklearn.metrics import roc_auc_score, average_precision_score, precision_score, recall_score, f1_score

# 账户级真实标签：按 card_id 聚合明细标签，这里用 max 规则（账户内只要有一条是 1，就认为账户为 1）
card_label = (
    df.groupby('card_id')['label']
    .max()
    .reindex(result_df['card_id'])  # 按 result_df 对齐
)

# 转成 numpy 数组
y_true = card_label.values.astype(int)

# OCSVM 的 ocsvm_score 越小越异常，
# 为了让“越大越可疑”更直观，这里取负号作为异常分数
anomaly_prob = -result_df['ocsvm_score'].values

# AUC-ROC
auc = roc_auc_score(y_true, anomaly_prob)

# PR-AUC（Average Precision）
pr_auc = average_precision_score(y_true, anomaly_prob)

# 二值预测阈值：用 ocsvm_label（-1=异常，1=正常）
y_pred = (result_df['ocsvm_label'].values == -1).astype(int)

precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

metrics = {
    'AUC': auc,
    'PR_AUC': pr_auc,
    'Precision': precision,
    'Recall': recall,
    'F1': f1,
}

metrics

{'AUC': 0.2639395591042403,
 'PR_AUC': 0.1301442779998414,
 'Precision': 0.06246514221974345,
 'Recall': 0.06285072951739619,
 'F1': 0.06265734265734266}